In [1]:
import sys
import os
import time
import random
import numpy as np
import tensorflow as tf 
from PIL import Image

SIZE = 400
WIDTH = 20
HEIGHT = 20
NUM_CLASSES = 31
iterations = 300

SAVER_DIR_P = "train-saver/province/"
SAVER_DIR_D = "train-saver/digits/"
SAVER_DIR_L = "train-saver/letters/"


PROVINCES = ("皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", 
             "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新")

LETTERS_DIGITS = ("0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","G","H","J","K",
                  "L","M","N","P","Q","R","S","T","U","V","W","X","Y","Z")

LETTERS = ("A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T",
                  "U","V","W","X","Y","Z")

license_num = ""

nProvinceIndex = 0

time_begin = time.time()

In [2]:
# 第一次遍历图片目录是为了获取图片总数
input_count = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/train/province/%s/' % i           # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            input_count += 1

# 定义对应维数和各维长度的数组
input_images = np.array([[0]*SIZE for i in range(input_count)])
input_labels = np.array([[0]*NUM_CLASSES for i in range(input_count)])

# 第二次遍历图片目录是为了生成图片数据和标签
index = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/train/province/%s/' % i          # 这里可以改成你自己的图片目录，i为分类标签
#         print ("省份简称是: %s" % PROVINCES[i])
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            filename = dir + filename
            img = Image.open(filename)
            width = img.size[0]
            height = img.size[1]
            for h in range(0, height):
                for w in range(0, width):
                    input_images[index][w+h*width] = img.getpixel((w, h))
#                 testimg = np.reshape(input_images[index], (20,20))
#                 print(testimg)
            input_labels[index][i] = 1
#                 print(input_labels[index])
            index += 1

# 第一次遍历图片目录是为了获取图片总数
val_count = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/val/province/%s/' % i           # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            val_count += 1

# 定义对应维数和各维长度的数组
val_images = np.array([[0]*SIZE for i in range(val_count)])
val_labels = np.array([[0]*NUM_CLASSES for i in range(val_count)])

# 第二次遍历图片目录是为了生成图片数据和标签
index = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/val/province/%s/' % i          # 这里可以改成你自己的图片目录，i为分类标签
#         print ("省份简称是: %s" % PROVINCES[i])
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            filename = dir + filename
            img = Image.open(filename)
            width = img.size[0]
            height = img.size[1]
            for h in range(0, height):
                for w in range(0, width):
                    val_images[index][w+h*width] = img.getpixel((w, h))
            val_labels[index][i] = 1
#                 print(i)
#                 print(val_labels[index])
            index += 1

In [3]:
def conv_layer(inputs, W, b, conv_strides, kernel_size, pool_strides, padding):
    L1_conv = tf.nn.conv2d(inputs, W, strides=conv_strides, padding=padding)
    L1_relu = tf.nn.relu(L1_conv + b)
    return tf.nn.max_pool(L1_relu, ksize=kernel_size, strides=pool_strides, padding='SAME')
 

# 定义全连接层函数
def full_connect(inputs, W, b):
    return tf.nn.relu(tf.matmul(inputs, W) + b)


x = tf.placeholder(tf.float32, shape=[None, SIZE])
y_ = tf.placeholder(tf.float32, shape=[None, NUM_CLASSES])

x_image = tf.reshape(x, [-1, WIDTH, HEIGHT, 1])

# 第一个卷积层
W_conv1 = tf.Variable(tf.truncated_normal([3, 3, 1, 16], stddev=0.1), name="W_conv1")
b_conv1 = tf.Variable(tf.constant(0.1, shape=[16]), name="b_conv1")
conv_strides = [1, 1, 1, 1]
kernel_size = [1, 2, 2, 1]
pool_strides = [1, 2, 2, 1]
L1_pool = conv_layer(x_image, W_conv1, b_conv1, conv_strides, kernel_size, pool_strides, padding='SAME')

# 第二个卷积层
W_conv2 = tf.Variable(tf.truncated_normal([3, 3, 16, 32], stddev=0.1), name="W_conv2")
b_conv2 = tf.Variable(tf.constant(0.1, shape=[32]), name="b_conv2")
conv_strides = [1, 1, 1, 1]
kernel_size = [1, 1, 1, 1]
pool_strides = [1, 1, 1, 1]
L2_pool = conv_layer(L1_pool, W_conv2, b_conv2, conv_strides, kernel_size, pool_strides, padding='SAME')

# 全连接层
W_fc1 = tf.Variable(tf.truncated_normal([10 * 10 * 32, 512], stddev=0.1), name="W_fc1")
b_fc1 = tf.Variable(tf.constant(0.1, shape=[512]), name="b_fc1")
h_pool2_flat = tf.reshape(L2_pool, [-1, 10 * 10 * 32])
h_fc1 = full_connect(h_pool2_flat, W_fc1, b_fc1)


# dropout
keep_prob = tf.placeholder(tf.float32)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


# readout层
W_fc2 = tf.Variable(tf.truncated_normal([512, NUM_CLASSES], stddev=0.1), name="W_fc2")
b_fc2 = tf.Variable(tf.constant(0.1, shape=[NUM_CLASSES]), name="b_fc2")


y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


# 定义优化器和训练op
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer((1e-4)).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
with tf.Session() as sess:
        # 初始化saver
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())

    time_elapsed = time.time() - time_begin
#     print("读取图片文件耗费时间：%d秒" % time_elapsed)
    time_begin = time.time()

#     print ("一共读取了 %s 个训练图像， %s 个标签" % (input_count, input_count))
#     print ("一共读取了 %s 个验证图像， %s 个标签" % (val_count, val_count))

#     print ("train shape:", (input_images.shape))
#     print ("val shape:", (val_images.shape))


    # 设置每次训练op的输入个数和迭代次数，这里为了支持任意图片总数，定义了一个余数remainder，譬如，如果每次训练op的输入个数为60，图片总数为150张，则前面两次各输入60张，最后一次输入30张（余数30）
    batch_size = 60
    iterations = iterations
    batches_count = int(input_count / batch_size)
    remainder = input_count % batch_size
#     print ("训练数据集分成 %s 批, 前面每批 %s 个数据，最后一批 %s 个数据" % (batches_count+1, batch_size, remainder))
#     print(input_images.shape)
#     print(val_images.shape)
    input_images.reshape((-1, 20*20))
    val_images.reshape((-1, 20*20))

    input_images = input_images.astype("float32")/255
    val_images = val_images.astype("float32")/255
    testimg = np.reshape(input_images[0], (-1, 20*20))
#         testimg = testimg.astype("float32")/255
#     print(testimg)
    testimg = np.reshape(val_images[0], (-1, 20*20))
#         testimg = testimg.astype("float32")/255
#     print(testimg)
#         print(type(input_images))
#         print(type(val_images))
#     print(input_images.shape)
#     print(val_images.shape)

    # 执行训练迭代
    for it in range(iterations):
        # 这里的关键是要把输入数组转为np.array
        for n in range(batches_count):
            train_step.run(feed_dict={x: input_images[n*batch_size:(n+1)*batch_size], y_: input_labels[n*batch_size:(n+1)*batch_size], keep_prob: 0.5})
        if remainder > 0:
            start_index = batches_count * batch_size;
            train_step.run(feed_dict={x: input_images[start_index:-1], y_: input_labels[start_index:-1], keep_prob: 0.5})

        # 每完成五次迭代，判断准确度是否已达到100%，达到则退出迭代循环
        iterate_accuracy = 0
        if it%5 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: input_images, y_: input_labels, keep_prob: 1.0})
            iterate_accuracy = accuracy.eval(feed_dict={x: val_images, y_: val_labels, keep_prob: 1.0})
            print ('第 %d 次训练迭代: train 准确率 %0.5f%%，val准确率 %0.5f%%' % (it, train_accuracy*100, iterate_accuracy*100))
            if iterate_accuracy >= 0.9999 and it >= 150:
                break;

#     print ('完成训练!')
    time_elapsed = time.time() - time_begin
    print ("训练耗费时间：%d秒" % time_elapsed)
    time_begin = time.time()

    # 保存训练结果
    if not os.path.exists(SAVER_DIR_P):
        print ('不存在训练数据保存目录，现在创建保存目录')
        os.makedirs(SAVER_DIR_P)
    saver_path = saver.save(sess, "%smodel.ckpt"%(SAVER_DIR_P))
    
    for n in range(1,2):
#             path = "test_images/%s.bmp" % (n)
        path = os.path.join("./test_images", "1510076148_823_1.bmp")
        print("\n预测：./test_images/1510076148_823_1.bmp")
        
        img = Image.open(path)
        img = img.resize((20,20))
        width = img.size[0]
        height = img.size[1]

        img_data = np.array([[0]*SIZE for i in range(1)])
        for h in range(0, height):
            for w in range(0, width):
                img_data[0][w+h*width]=img.getpixel((w,h))
#                     if img.getpixel((w, h)) < 190:
#                         img_data[0][w+h*width] = 1
#                     else:
#                         img_data[0][w+h*width] = 0

        img = img_data.astype('float32')/255
        result = sess.run(conv, feed_dict = {x: img, keep_prob: 1.0})
        max1 = 0
        max2 = 0
        max3 = 0
        max1_index = 0
        max2_index = 0
        max3_index = 0
        for j in range(NUM_CLASSES):
            if result[0][j] > max1:
                max1 = result[0][j]
                max1_index = j
                continue
            if (result[0][j]>max2) and (result[0][j]<=max1):
                max2 = result[0][j]
                max2_index = j
                continue
            if (result[0][j]>max3) and (result[0][j]<=max2):
                max3 = result[0][j]
                max3_index = j
                continue

        nProvinceIndex = max1_index
        print ("概率：  [%s %0.2f%%]    [%s %0.2f%%]    [%s %0.2f%%]" % (PROVINCES[max1_index],max1*100, PROVINCES[max2_index],max2*100, PROVINCES[max3_index],max3*100))

    print ("省份简称是: %s" % PROVINCES[nProvinceIndex])

第 0 次训练迭代: train 准确率 15.72508%，val准确率 13.68421%
第 5 次训练迭代: train 准确率 43.03706%，val准确率 37.63158%
第 10 次训练迭代: train 准确率 58.94207%，val准确率 56.31579%
第 15 次训练迭代: train 准确率 70.92479%，val准确率 70.26316%
第 20 次训练迭代: train 准确率 80.71249%，val准确率 80.78948%
第 25 次训练迭代: train 准确率 85.57035%，val准确率 84.21053%
第 30 次训练迭代: train 准确率 88.70097%，val准确率 87.89474%
第 35 次训练迭代: train 准确率 91.43577%，val准确率 90.00000%
第 40 次训练迭代: train 准确率 93.19899%，val准确率 91.84210%
第 45 次训练迭代: train 准确率 94.60238%，val准确率 92.63158%
第 50 次训练迭代: train 准确率 95.46599%，val准确率 93.68421%
第 55 次训练迭代: train 准确率 96.18568%，val准确率 94.73684%
第 60 次训练迭代: train 准确率 96.65347%，val准确率 94.47368%
第 65 次训练迭代: train 准确率 97.26520%，val准确率 95.00000%
第 70 次训练迭代: train 准确率 97.98489%，val准确率 95.26316%
第 75 次训练迭代: train 准确率 98.45268%，val准确率 95.26316%
第 80 次训练迭代: train 准确率 98.92048%，val准确率 95.00000%
第 85 次训练迭代: train 准确率 99.17237%，val准确率 95.00000%
第 90 次训练迭代: train 准确率 99.31630%，val准确率 95.78947%
第 95 次训练迭代: train 准确率 99.38827%，val准确率 96.31579%
第 100 次训练迭代: train 准确率

In [10]:
# 第一次遍历图片目录是为了获取图片总数
input_count = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/train/letter/%s/' % i           # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            input_count += 1

# 定义对应维数和各维长度的数组
input_images = np.array([[0]*SIZE for i in range(input_count)])
input_labels = np.array([[0]*NUM_CLASSES for i in range(input_count)])

# 第二次遍历图片目录是为了生成图片数据和标签
index = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/train/letter/%s/' % i          # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            filename = dir + filename
            img = Image.open(filename)
            width = img.size[0]
            height = img.size[1]
            for h in range(0, height):
                for w in range(0, width):
                    input_images[index][w+h*width] = img.getpixel((w,h))
            input_labels[index][i] = 1
            index += 1

# 第一次遍历图片目录是为了获取图片总数
val_count = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/val/letter/%s/' % i           # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            val_count += 1

# 定义对应维数和各维长度的数组
val_images = np.array([[0]*SIZE for i in range(val_count)])
val_labels = np.array([[0]*NUM_CLASSES for i in range(val_count)])

# 第二次遍历图片目录是为了生成图片数据和标签
index = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/val/letter/%s/' % i          # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            filename = dir + filename
            img = Image.open(filename)
            width = img.size[0]
            height = img.size[1]
            for h in range(0, height):
                for w in range(0, width):
                    val_images[index][w+h*width] = img.getpixel((w,h))
            val_labels[index][i] = 1
            index += 1

In [11]:
# 第一个卷积层
W_conv1 = tf.Variable(tf.truncated_normal([3, 3, 1, 16], stddev=0.1), name="W_conv1")
b_conv1 = tf.Variable(tf.constant(0.1, shape=[16]), name="b_conv1")
conv_strides = [1, 1, 1, 1]
kernel_size = [1, 2, 2, 1]
pool_strides = [1, 2, 2, 1]
L1_pool = conv_layer(x_image, W_conv1, b_conv1, conv_strides, kernel_size, pool_strides, padding='SAME')

# 第二个卷积层
W_conv2 = tf.Variable(tf.truncated_normal([3, 3, 16, 32], stddev=0.1), name="W_conv2")
b_conv2 = tf.Variable(tf.constant(0.1, shape=[32]), name="b_conv2")
conv_strides = [1, 1, 1, 1]
kernel_size = [1, 1, 1, 1]
pool_strides = [1, 1, 1, 1]
L2_pool = conv_layer(L1_pool, W_conv2, b_conv2, conv_strides, kernel_size, pool_strides, padding='SAME')


# 全连接层
W_fc1 = tf.Variable(tf.truncated_normal([10 * 10 * 32, 512], stddev=0.1), name="W_fc1")
b_fc1 = tf.Variable(tf.constant(0.1, shape=[512]), name="b_fc1")
h_pool2_flat = tf.reshape(L2_pool, [-1, 10 * 10 * 32])
h_fc1 = full_connect(h_pool2_flat, W_fc1, b_fc1)


# dropout
keep_prob = tf.placeholder(tf.float32)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


# readout层
W_fc2 = tf.Variable(tf.truncated_normal([512, NUM_CLASSES], stddev=0.1), name="W_fc2")
b_fc2 = tf.Variable(tf.constant(0.1, shape=[NUM_CLASSES]), name="b_fc2")

# 定义优化器和训练op
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer((1e-4)).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [12]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
 
    time_elapsed = time.time() - time_begin
#     print("读取图片文件耗费时间：%d秒" % time_elapsed)
    time_begin = time.time()

#     print ("一共读取了 %s 个train图像， %s 个标签" % (input_count, input_count))
#     print ("一共读取了 %s 个val图像， %s 个标签" % (val_count, val_count))

#     print("train shape:", (input_images.shape))
#     print("val shape", (val_images.shape))


    # 设置每次训练op的输入个数和迭代次数，这里为了支持任意图片总数，定义了一个余数remainder，譬如，如果每次训练op的输入个数为60，图片总数为150张，则前面两次各输入60张，最后一次输入30张（余数30）
    batch_size = 60
    iterations = iterations
    batches_count = int(input_count / batch_size)
    remainder = input_count % batch_size
#     print ("训练数据集分成 %s 批, 前面每批 %s 个数据，最后一批 %s 个数据" % (batches_count+1, batch_size, remainder))

    input_images = input_images.astype("float32")/255
    val_images = val_images.astype("float32")/255
    testimg = np.reshape(input_images[0], (-1, 20*20))
#     print(testimg)
    testimg = np.reshape(val_images[0], (-1, 20*20))
#     print(testimg)

    # 执行训练迭代
    for it in range(iterations):
        # 这里的关键是要把输入数组转为np.array
        for n in range(batches_count):
            train_step.run(feed_dict={x: input_images[n*batch_size:(n+1)*batch_size], y_: input_labels[n*batch_size:(n+1)*batch_size], keep_prob: 0.5})
        if remainder > 0:
            start_index = batches_count * batch_size;
            train_step.run(feed_dict={x: input_images[start_index:input_count-1], y_: input_labels[start_index:input_count-1], keep_prob: 0.5})

        # 每完成五次迭代，判断准确度是否已达到100%，达到则退出迭代循环
        iterate_accuracy = 0
        if it%5 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: input_images, y_: input_labels, keep_prob:1.0})
            iterate_accuracy = accuracy.eval(feed_dict={x: val_images, y_: val_labels, keep_prob: 1.0})
            print ('第 %d 次训练迭代: train 准确率 %0.5f%%, val准确率 %0.5f%%' % (it, train_accuracy*100, iterate_accuracy*100))
            if iterate_accuracy >= 0.9999 and it >= iterations:
                break;

#     print ('完成训练!')
    time_elapsed = time.time() - time_begin
    print ("训练耗费时间：%d秒" % time_elapsed)
    time_begin = time.time()

    # 保存训练结果
    if not os.path.exists(SAVER_DIR_D):
        print ('不存在训练数据保存目录，现在创建保存目录')
        os.makedirs(SAVER_DIR_D)
    # 初始化saver
    saver = tf.train.Saver()            
    saver_path = saver.save(sess, "%smodel.ckpt"%(SAVER_DIR_D))

    for n in range(3,8):
        path = "test_images/%s.bmp" % (n)
        img = Image.open(path)
        width = img.size[0]
        height = img.size[1]

        img_data = [[0]*SIZE for i in range(1)]
        for h in range(0, height):
            for w in range(0, width):
                if img.getpixel((w, h)) < 190:
                    img_data[0][w+h*width] = 1
                else:
                    img_data[0][w+h*width] = 0

        result = sess.run(conv, feed_dict = {x: np.array(img_data), keep_prob: 1.0})

        max1 = 0
        max2 = 0
        max3 = 0
        max1_index = 0
        max2_index = 0
        max3_index = 0
        for j in range(NUM_CLASSES):
            if result[0][j] > max1:
                max1 = result[0][j]
                max1_index = j
                continue
            if (result[0][j]>max2) and (result[0][j]<=max1):
                max2 = result[0][j]
                max2_index = j
                continue
            if (result[0][j]>max3) and (result[0][j]<=max2):
                max3 = result[0][j]
                max3_index = j
                continue

        license_num = license_num + LETTERS_DIGITS[max1_index]
        print ("概率：  [%s %0.2f%%]    [%s %0.2f%%]    [%s %0.2f%%]" % (LETTERS_DIGITS[max1_index],max1*100, LETTERS_DIGITS[max2_index],max2*100, LETTERS_DIGITS[max3_index],max3*100))

    print ("车牌编号是: 【%s】" % license_num)

第 0 次训练迭代: train 准确率 12.10280%, val准确率 6.70391%
第 5 次训练迭代: train 准确率 63.32554%, val准确率 28.66165%
第 10 次训练迭代: train 准确率 82.05608%, val准确率 57.42045%
第 15 次训练迭代: train 准确率 90.10903%, val准确率 65.38742%
第 20 次训练迭代: train 准确率 95.97352%, val准确率 72.91718%
第 25 次训练迭代: train 准确率 97.35981%, val准确率 75.51615%
第 30 次训练迭代: train 准确率 98.27103%, val准确率 76.14768%
第 35 次训练迭代: train 准确率 98.90966%, val准确率 77.11926%
第 40 次训练迭代: train 准确率 99.05763%, val准确率 78.57664%
第 45 次训练迭代: train 准确率 99.29128%, val准确率 79.71824%
第 50 次训练迭代: train 准确率 99.53271%, val准确率 79.54822%
第 55 次训练迭代: train 准确率 99.56386%, val准确率 79.40248%
第 60 次训练迭代: train 准确率 99.81309%, val准确率 78.84382%
第 65 次训练迭代: train 准确率 99.78193%, val准确率 79.52393%
第 70 次训练迭代: train 准确率 99.88317%, val准确率 79.64537%
第 75 次训练迭代: train 准确率 99.89097%, val准确率 80.15545%
第 80 次训练迭代: train 准确率 99.73520%, val准确率 80.03401%
第 85 次训练迭代: train 准确率 99.92990%, val准确率 79.11100%
第 90 次训练迭代: train 准确率 99.95327%, val准确率 76.51202%
第 95 次训练迭代: train 准确率 99.40810%, val准确率 80.00972%
第 1

IndexError: list assignment index out of range

In [13]:
# 第一次遍历图片目录是为了获取图片总数
input_count = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/train/area/%s/' % i           # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            input_count += 1

# 定义对应维数和各维长度的数组
input_images = np.array([[0]*SIZE for i in range(input_count)])
input_labels = np.array([[0]*NUM_CLASSES for i in range(input_count)])

# 第二次遍历图片目录是为了生成图片数据和标签
index = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/train/area/%s/' % i          # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            filename = dir + filename
            img = Image.open(filename)
            width = img.size[0]
            height = img.size[1]
            for h in range(0, height):
                for w in range(0, width):
                    input_images[index][w+h*width] = img.getpixel((w,h))
            input_labels[index][i] = 1
            index += 1

# 第一次遍历图片目录是为了获取图片总数
val_count = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/val/area/%s/' % i           # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            val_count += 1

# 定义对应维数和各维长度的数组
val_images = np.array([[0]*SIZE for i in range(val_count)])
val_labels = np.array([[0]*NUM_CLASSES for i in range(val_count)])

# 第二次遍历图片目录是为了生成图片数据和标签
index = 0
for i in range(0,NUM_CLASSES):
    dir = './dataset/val/area/%s/' % i          # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            filename = dir + filename
            img = Image.open(filename)
            width = img.size[0]
            height = img.size[1]
            for h in range(0, height):
                for w in range(0, width):
                    val_images[index][w+h*width] = img.getpixel((w, h))
            val_labels[index][i] = 1
            index += 1

In [14]:
# 第一个卷积层
W_conv1 = tf.Variable(tf.truncated_normal([3, 3, 1, 16], stddev=0.1), name="W_conv1")
b_conv1 = tf.Variable(tf.constant(0.1, shape=[16]), name="b_conv1")
conv_strides = [1, 1, 1, 1]
kernel_size = [1, 2, 2, 1]
pool_strides = [1, 2, 2, 1]
L1_pool = conv_layer(x_image, W_conv1, b_conv1, conv_strides, kernel_size, pool_strides, padding='SAME')

# 第二个卷积层
W_conv2 = tf.Variable(tf.truncated_normal([3, 3, 16, 32], stddev=0.1), name="W_conv2")
b_conv2 = tf.Variable(tf.constant(0.1, shape=[32]), name="b_conv2")
conv_strides = [1, 1, 1, 1]
kernel_size = [1, 1, 1, 1]
pool_strides = [1, 1, 1, 1]
L2_pool = conv_layer(L1_pool, W_conv2, b_conv2, conv_strides, kernel_size, pool_strides, padding='SAME')


# 全连接层
W_fc1 = tf.Variable(tf.truncated_normal([10 * 10 * 32, 512], stddev=0.1), name="W_fc1")
b_fc1 = tf.Variable(tf.constant(0.1, shape=[512]), name="b_fc1")
h_pool2_flat = tf.reshape(L2_pool, [-1, 10 * 10 * 32])
h_fc1 = full_connect(h_pool2_flat, W_fc1, b_fc1)


# dropout
keep_prob = tf.placeholder(tf.float32)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


# readout层
W_fc2 = tf.Variable(tf.truncated_normal([512, NUM_CLASSES], stddev=0.1), name="W_fc2")
b_fc2 = tf.Variable(tf.constant(0.1, shape=[NUM_CLASSES]), name="b_fc2")

# 定义优化器和训练op
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer((1e-4)).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
 
    time_elapsed = time.time() - time_begin
#     print("读取图片文件耗费时间：%d秒" % time_elapsed)
    time_begin = time.time()

#     print ("一共读取了 %s 个训练图像， %s 个标签" % (input_count, input_count))
#     print ("一共读取了 %s 个验证图像， %s 个标签" % (val_count, val_count))

#     print ("train shape:", (input_images.shape))
#     print ("val shape:", (val_images.shape))

    # 设置每次训练op的输入个数和迭代次数，这里为了支持任意图片总数，定义了一个余数remainder，譬如，如果每次训练op的输入个数为60，图片总数为150张，则前面两次各输入60张，最后一次输入30张（余数30）
    batch_size = 60
    iterations = iterations
    batches_count = int(input_count / batch_size)
    remainder = input_count % batch_size
#     print ("训练数据集分成 %s 批, 前面每批 %s 个数据，最后一批 %s 个数据" % (batches_count+1, batch_size, remainder))
#     print(input_images.shape)
#     print(val_images.shape)
    input_images.reshape((-1, 20*20))
    val_images.reshape((-1, 20*20))

    input_images = input_images.astype("float32")/255
    val_images = val_images.astype("float32")/255
    testimg = np.reshape(input_images[0], (-1, 20*20))
#     print(testimg)
    testimg = np.reshape(val_images[0], (-1, 20*20))
#     print(testimg)
#     print(input_images.shape)
#     print(val_images.shape)

    # 执行训练迭代
    for it in range(iterations):
        # 这里的关键是要把输入数组转为np.array
        for n in range(batches_count):
            train_step.run(feed_dict={x: input_images[n*batch_size:(n+1)*batch_size], y_: input_labels[n*batch_size:(n+1)*batch_size], keep_prob: 0.5})
        if remainder > 0:
            start_index = batches_count * batch_size;
            train_step.run(feed_dict={x: input_images[start_index:input_count-1], y_: input_labels[start_index:input_count-1], keep_prob: 0.5})

        # 每完成五次迭代，判断准确度是否已达到100%，达到则退出迭代循环
        iterate_accuracy = 0
        if it%5 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: input_images, y_: input_labels, keep_prob: 1.0})
            iterate_accuracy = accuracy.eval(feed_dict={x: val_images, y_: val_labels, keep_prob: 1.0})
            print ('第 %d 次训练迭代: train准确率 %0.5f%%, val准确率 %0.5f%%' % (it, train_accuracy*100, iterate_accuracy*100))
            if iterate_accuracy >= 0.9999 and it >= iterations:
                break;

#     print ('完成训练!')
    time_elapsed = time.time() - time_begin
    print ("训练耗费时间：%d秒" % time_elapsed)
    time_begin = time.time()

    # 保存训练结果
    if not os.path.exists(SAVER_DIR_L):
        print ('不存在训练数据保存目录，现在创建保存目录')
        os.makedirs(SAVER_DIR_L)
    # 初始化saver
    saver = tf.train.Saver()            
    saver_path = saver.save(sess, "%smodel.ckpt"%(SAVER_DIR_L))

    for n in range(1, 2):
        #path = "test_images/%s.bmp" % (n)
        path = os.path.join("./test_images", "1510076148_824_3.bmp")
        print("\n预测：./test_images/1510076148_824_3.bmp")
        img = Image.open(path)
        img = img.resize((20,20))
        width = img.size[0]
        height = img.size[1]

        img_data = np.array([[0]*SIZE for i in range(1)])
        for h in range(0, height):
            for w in range(0, width):
                img_data[0][w+h*width] = img.getpixel((w,h))

        img = img_data.astype('float32')/255

        result = sess.run(conv, feed_dict = {x: img, keep_prob: 1.0})

        max1 = 0
        max2 = 0
        max3 = 0
        max1_index = 0
        max2_index = 0
        max3_index = 0
        for j in range(NUM_CLASSES):
            if result[0][j] > max1:
                max1 = result[0][j]
                max1_index = j
                continue
            if (result[0][j]>max2) and (result[0][j]<=max1):
                max2 = result[0][j]
                max2_index = j
                continue
            if (result[0][j]>max3) and (result[0][j]<=max2):
                max3 = result[0][j]
                max3_index = j
                continue

        if n == 3:
            license_num += "-"
        license_num = license_num + LETTERS[max1_index]
        print ("概率：  [%s %0.2f%%]    [%s %0.2f%%]    [%s %0.2f%%]" % (LETTERS[max1_index],max1*100, LETTERS[max2_index],max2*100, LETTERS[max3_index],max3*100))

    print ("城市代号是: 【%s】" % license_num)

第 0 次训练迭代: train准确率 17.87576%, val准确率 7.06663%
第 5 次训练迭代: train准确率 66.31908%, val准确率 30.48745%
第 10 次训练迭代: train准确率 88.44034%, val准确率 63.42660%
第 15 次训练迭代: train准确率 95.29272%, val准确率 73.40640%
第 20 次训练迭代: train准确率 95.79920%, val准确率 77.58870%
第 25 次训练迭代: train准确率 96.02264%, val准确率 80.09807%
第 30 次训练迭代: train准确率 96.40995%, val准确率 79.75194%
第 35 次训练迭代: train准确率 97.12498%, val准确率 79.95385%
第 40 次训练迭代: train准确率 98.06346%, val准确率 79.80964%
第 45 次训练迭代: train准确率 98.79339%, val准确率 79.55005%
第 50 次训练迭代: train准确率 99.29987%, val准确率 79.31930%
第 55 次训练迭代: train准确率 99.44883%, val准确率 80.06923%
第 60 次训练迭代: train准确率 99.67228%, val准确率 79.63657%
第 65 次训练迭代: train准确率 98.27201%, val准确率 75.88693%
第 70 次训练迭代: train准确率 99.85104%, val准确率 81.42486%
第 75 次训练迭代: train准确率 99.88083%, val准确率 80.35766%
第 80 次训练迭代: train准确率 99.65738%, val准确率 78.02134%
第 85 次训练迭代: train准确率 99.92552%, val准确率 81.25180%
第 90 次训练迭代: train准确率 99.86593%, val准确率 79.37698%
第 95 次训练迭代: train准确率 99.12111%, val准确率 77.79059%
第 100 次训练迭代: train准确率 9